# Subscriber
**Run this for being a Subscriber**   
Subscribe to **topic: "CPE_HOUSE/room1"**   

Clients can subscribe to the exact topic of a published message or use wildcards to subscribe to multiple topics at the same time <br>
A wildcard can only be used to subscribe to topics, not to publish a message


Test publish from this site:
https://www.hivemq.com/demos/websocket-client/

and run code below for see the result

In [2]:
import paho.mqtt.client as mqtt
MQTT_HOST = "broker.mqttdashboard.com"
MQTT_PORT = 1883
#MQTT_HOST = "mqtt-dashboard.com"
#MQTT_PORT = 8884
MQTT_KEEPALIVE_INTERVAL = 45 #45 seconds
MQTT_TOPIC = "CPE_DEMO_HOUSE/#"

def on_connect(self,client, userdata, rc):
    print("MQTT Connected.")
    self.subscribe(MQTT_TOPIC)
    # add more subscribe ...
    # e.g.;
    # self.subscribe("CPE_DEMO_HOUSE/room2/#")

def on_message(client, userdata, msg):
    print("message received...")
    print(msg.topic + " "+ str(msg.qos) + " " + str(msg.payload.decode("utf-8","strict")))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.connect(MQTT_HOST)
client.loop_forever()

MQTT Connected.
message received...
CPE_DEMO_HOUSE/ 0 {"Temperature": 32, "Humidity": 54, "Light": 102}
message received...
CPE_DEMO_HOUSE/Hill 0 {"Temperature": 20, "Humidity": 30, "Light": 200}
message received...
CPE_DEMO_HOUSE/room 0 {"Temperature": 32, "Humidity": 54, "Light": 102}
message received...
CPE_DEMO_HOUSE/12 0 {"Temperature": 32, "Humidity": 54, "Light": 102}
message received...
CPE_DEMO_HOUSE/BACKROOMS 0 OUR GROUP IS
message received...
CPE_DEMO_HOUSE/room2 0 {"Temperature": 20, "Humidity": 30, "Light": 200}
message received...
CPE_DEMO_HOUSE/clowns 0 {"Temperature": 69, "Humidity": 96, "Light": 39}
message received...
CPE_DEMO_HOUSE/ARM 0 your
message received...
CPE_DEMO_HOUSE/room1 0 {"Temperature": 35, "Humidity": 60, "Light": 138}
message received...
CPE_DEMO_HOUSE/r 0 {"Temperature": 32, "Humidity": 54, "Light": 102}
message received...
CPE_DEMO_HOUSE/room4 0 ARM
message received...
CPE_DEMO_HOUSE/FOREST 0 HELP ME
message received...
CPE_DEMO_HOUSE/room3 0 OUR GR

KeyboardInterrupt: 

In [ ]:
import paho.mqtt.client as mqtt
import csv
import json
from datetime import datetime
import portalocker

MQTT_HOST = "broker.mqttdashboard.com"
MQTT_PORT = 1883
MQTT_KEEPALIVE_INTERVAL = 45 #45 seconds
MQTT_TOPIC = "CPE_DEMO_HOUSE/#"
CSV_FILE = "sensor_data.csv"

def on_connect(self,client, userdata, rc):
    print("MQTT Connected.")
    self.subscribe(MQTT_TOPIC)
    # add more subscribe ...
    # e.g.;
    # self.subscribe("CPE_DEMO_HOUSE/room2/#")

def on_message(client, userdata, msg):
    print("message received...")
    print(msg.topic + " "+ str(msg.qos) + " " + str(msg.payload.decode("utf-8","strict")))

     # Try parsing the message payload as JSON
    try:
        data = json.loads(msg.payload.decode("utf-8"))
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        topic = msg.topic
        temperature = data.get("Temperature", "N/A")
        humidity = data.get("Humidity", "N/A")
        light = data.get("Light", "N/A")

        # Open CSV in append mode and write the data
        with open(CSV_FILE, mode='a', newline='') as file:
            # Acquire file lock
            portalocker.lock(file, portalocker.LOCK_EX)
            
            writer = csv.writer(file)
            # Write headers if the file is empty
            if file.tell() == 0:
                writer.writerow(["Timestamp", "Topic", "Temperature", "Humidity", "Light"])
            # Write the data row
            writer.writerow([timestamp, topic, temperature, humidity, light])
            # Release file lock
            portalocker.unlock(file)

        print(f"Data written to {CSV_FILE}: {timestamp}, {topic}, {temperature}, {humidity}, {light}")

    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON: {e}")

client = mqtt.Client()

client.on_connect = on_connect
client.on_message = on_message

client.connect(MQTT_HOST)
client.loop_forever()

MQTT Connected.
message received...
CPE_DEMO_HOUSE/ 0 {"Temperature": 32, "Humidity": 54, "Light": 102}
Data written to sensor_data.csv: 2024-09-24 11:28:22, CPE_DEMO_HOUSE/, 32, 54, 102
message received...
CPE_DEMO_HOUSE/Hill 0 {"Temperature": 20, "Humidity": 30, "Light": 200}
Data written to sensor_data.csv: 2024-09-24 11:28:22, CPE_DEMO_HOUSE/Hill, 20, 30, 200
message received...
CPE_DEMO_HOUSE/room 0 {"Temperature": 32, "Humidity": 54, "Light": 102}
Data written to sensor_data.csv: 2024-09-24 11:28:22, CPE_DEMO_HOUSE/room, 32, 54, 102
message received...
CPE_DEMO_HOUSE/12 0 {"Temperature": 32, "Humidity": 54, "Light": 102}
Data written to sensor_data.csv: 2024-09-24 11:28:22, CPE_DEMO_HOUSE/12, 32, 54, 102
message received...
CPE_DEMO_HOUSE/BACKROOMS 0 OUR GROUP IS
Failed to decode JSON: Expecting value: line 1 column 1 (char 0)
message received...
CPE_DEMO_HOUSE/room2 0 {"Temperature": 20, "Humidity": 30, "Light": 200}
Data written to sensor_data.csv: 2024-09-24 11:28:22, CPE_DEMO